# Data Hygiene

In [1]:
import glob
import os
import pandas as pd   
from datetime import datetime, date

sen = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "sentencing00000000000?.csv"))))
len(sen)

def age(born, sen_date):
    return sen_date.year - born.year - ((sen_date.month, sen_date.day) < (born.month, born.day))

/Users/robert.ling/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (25,89,90,98,99,107,108,116,117,125,126,134,135,143,144,152,153,161,162,170,171,186,187,194,195,202,203,210,211,218,219,226,227,234,235,242,243,250,251,271) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/robert.ling/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (25,89,90,98,99,107,108,116,117,125,126,134,135,143,144,152,153,161,162,170,171,178,179,186,187,194,195,202,203,210,211,218,219,226,227,234,235,242,243,250,251,271) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/robert.ling/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (9,16,19,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39

In [2]:
sen = sen[sen['race']!='RACE'] #drop the header line
sen['spyrs'] = pd.to_numeric(sen['spyrs'], errors='coerce')
sen['spmths'] = pd.to_numeric(sen['spmths'], errors='coerce')
sen['spdays'] = pd.to_numeric(sen['spdays'], errors='coerce')
sen['sp_total_days'] = (sen['spyrs']*365) + (sen['spmths']*30) + sen['spdays']
sen['totpts'] = pd.to_numeric(sen['totpts'],errors='coerce')
sen['dob'] = pd.to_datetime(sen['dob'], errors="coerce")
sen['sentdte'] = pd.to_datetime(sen['sentdte'], errors="coerce")
sen['age'] = sen.apply(lambda x: age(x.dob, x.sentdte), axis=1)
sen['offdte'] = pd.to_datetime(sen['offdte'], errors="coerce")
sen['haboff'] = sen['haboff'].map({'YES': True, 'NO': False}) 
sen['habvio'] = sen['habvio'].map({'YES': True, 'NO': False}) 
sen["county"] = sen["county"].str.lower()

In [11]:
#stop the noise
sen1 = sen[["fstnm", "lstnm", "age", "race", "gender", "county", "totpts", "judge", "clfely", "statut", "offlvl", "offdte", \
            "sp_total_days", "enhanc", "haboff", "habvio", "life", "vcc", "prrpact", "dornum", \
            "prflst1","prfely1", "prflst2","prfely2", "prflst3","prfely3", "prflst4","prfely4", "sentdte"]]

In [12]:
# Join in FIPs for Maps
fips = pd.read_csv("./fips-codes/state_and_county_fips_master.csv")
fips = fips[fips.state=="FL"]
fips["county"] = fips["county"].str.lower()
fips.head() #"bop exchange and 00-interstate are in DOC-sentencing"

sen1 = pd.merge(sen1, fips, on='county', how='left')

## Run some quick tests

In [13]:
sen1.dtypes  #sanity check types are nice-correct to query/work with

fstnm                    object
lstnm                    object
age                       int64
race                     object
gender                   object
county                   object
totpts                  float64
judge                    object
clfely                   object
statut                   object
offlvl                   object
offdte           datetime64[ns]
sp_total_days             int64
enhanc                   object
haboff                     bool
habvio                     bool
life                     object
vcc                      object
prrpact                  object
dornum                   object
prflst1                  object
prfely1                  object
prflst2                  object
prfely2                  object
prflst3                  object
prfely3                  object
prflst4                  object
prfely4                  object
sentdte          datetime64[ns]
fips                      int64
state                    object
dtype: o

In [14]:
sen1[sen1.fips.isnull()]       #All rows have a fips code

,fstnm,lstnm,age,race,gender,county,totpts,judge,clfely,statut,...,prfely1,prflst2,prfely2,prflst3,prfely3,prflst4,prfely4,sentdte,fips,state


In [ ]:
sen1.dornum.value_counts()

In [15]:
sen1[sen1.fstnm.isnull() & sen1.lstnm.isnull()]    #make sure no defendents are without fstnm, lstnm

,fstnm,lstnm,age,race,gender,county,totpts,judge,clfely,statut,...,prfely1,prflst2,prfely2,prflst3,prfely3,prflst4,prfely4,sentdte,fips,state


In [16]:
sen1.to_csv("sentencing_s.csv", index=False)
#use Azure Data Studio to import this into MSSQL, allow nulls all fields
#or copy over to S3 bucket to query with Athena

# Regression Modelling

In [ ]:
import pandas as pd
from pycaret.regression import *
from datetime import datetime, date
%matplotlib inline

sen1 = pd.read_csv("sentencing_s.csv")
#sen1 = sen1.sample(frac=.05)
#len(sen1)

In [ ]:
s = setup(sen1, target="totpts",  session_id=123, numeric_features=["age"], categorical_features=["lstnm", "age", "race", "gender", "judge"])

In [ ]:
eda()